## 测试代理是否正常

In [1]:
# 测试代理
import os
import requests

# 设置代理环境变量
#os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
#os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
#os.environ['ALL_PROXY'] = 'socks5://127.0.0.1:7891'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 测试代理连接
try:
    response = requests.get('https://huggingface.co', timeout=10)
    print("✅ HuggingFace 连接成功，状态码:", response.status_code)
    print("Test server is ubuntu22.04 GPU 2080Ti 22G")
except Exception as e:
    print("连接失败:", e)

# 设置 HuggingFace 缓存路径
os.environ['HF_HOME'] = '/home/KevinLiangX/Codes/LLM-quickstart-main/hf'
os.environ['HF_HUB_CACHE'] = '/home/KevinLiangX/Codes/LLM-quickstart-main/hf_hu'


# 服务器环境 ubuntu22.04 GPU 2080Ti 22G

连接失败: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7a7f85ee3700>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


In [2]:
# 定义全局变量和参数
model_name_or_path = 'ChatGLM3-6b'  # 模型ID或本地路径
train_data_path = 'HasturOfficial/adgen'    # 训练数据路径
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 4                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
resume_from_checkpoint = None             # 如果从checkpoint恢复训练，指定路径
prompt_text = ''                          # 所有数据前的指令文本
compute_dtype = 'fp16'                    # 计算数据类型（fp32, fp16, bf16）

In [3]:
from datasets import load_dataset

dataset = load_dataset(train_data_path)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 114599
    })
    validation: Dataset({
        features: ['content', 'summary'],
        num_rows: 1070
    })
})

In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"], num_examples=3)

,content,summary
0,类型#裙*图案#线条*裙衣长#中长款*裙款式#抽绳,这款裙子衬托出满满的仙气，正适合想要年轻，又有女人味的你，优雅减龄。腰部抽绳设计，可以随意调节，巧妙优化身材比例，勾勒窈窕腰身，中长款的版型设计，可以有效的修饰腿部线条。
1,类型#上衣*版型#显瘦*风格#欧美*风格#清新*风格#潮*图案#字母*图案#文字*图案#印花*衣样式#卫衣*衣款式#连帽,生活中既想让自己看起来，显瘦苗条又更想展示出自身的潮流感，这件卫衣完美的体现出了这种风格。高档制作工艺的欧美字母印花图案，看起来清新又时髦。采用的连帽设计，可以抵挡住寒风的侵袭。用来拉长身材的高腰版型，不再担心自己矮小拖沓。
2,类型#裙*版型#宽松*材质#棉*颜色#灰色*裙下摆#毛边*裙长#连衣裙,这款来自BRAND的连衣裙采用了独特的气球廓形剪裁，凸显出十足的设计感，再结合柔软的复合空气棉，不仅富有更多的包容性，帮你隐藏多余的肉肉，还带给你宽松舒适的穿着感受。而且还采用了别致的<UNK>毛边处理，展现出与众不同的格调，再搭配沉稳温和的灰色调，打造出独具一格的穿着风格。


In [7]:
# 使用 ChatGLM3-6b Tokenizer 处理数据
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [8]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

In [9]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

Map:   0%|          | 0/114599 [00:00<?, ? examples/s]

In [10]:
show_random_elements(tokenized_dataset, num_examples=1)

,input_ids,labels
0,"[64790, 64792, 30910, 33467, 31010, 49534, 30998, 37505, 31010, 37216, 30998, 55500, 46025, 31010, 42373, 30998, 55500, 46025, 31010, 44630, 30998, 55500, 54811, 58709, 31010, 54716, 56241, 30910, 33730, 40697, 54530, 44630, 54603, 42373, 31123, 54716, 54847, 54716, 56241, 54530, 54811, 58709, 31123, 37216, 40315, 46201, 31123, 35405, 59604, 57263, 33550, 31123, 31701, 54849, 54533, 57342, 54607, 55166, 34678, 56059, 54530, 31155, 31799, 32745, 38871, 32789, 54811, 58709, 37216, 54706, 35212, 31123, 54657, 55101, 33454, 54733, 55014, 31155, 35997, 42128, 54530, 54811, 58709, 31123, 33454, 31799, 31123, 40697, 54999, 55583, 31123, 45095, 38589, 31155, 2]","[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 33730, 40697, 54530, 44630, 54603, 42373, 31123, 54716, 54847, 54716, 56241, 54530, 54811, 58709, 31123, 37216, 40315, 46201, 31123, 35405, 59604, 57263, 33550, 31123, 31701, 54849, 54533, 57342, 54607, 55166, 34678, 56059, 54530, 31155, 31799, 32745, 38871, 32789, 54811, 58709, 37216, 54706, 35212, 31123, 54657, 55101, 33454, 54733, 55014, 31155, 35997, 42128, 54530, 54811, 58709, 31123, 33454, 31799, 31123, 40697, 54999, 55583, 31123, 45095, 38589, 31155, 2]"


In [11]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)

In [12]:
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/114599 [00:00<?, ? examples/s]

In [13]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [14]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

In [15]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

In [17]:
from huggingface_hub import snapshot_download
from transformers import AutoModel

# 先下载模型（支持断点续传）
#model_path = snapshot_download(
#    repo_id=model_name_or_path,
#    revision='b098244',
#    resume_download=True,  # 启用断点续传
#    local_files_only=False,
#    force_download=False,
   # max_workers=4,
#)

# 然后加载模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)



#model = AutoModel.from_pretrained(model_name_or_path,
#                                  quantization_config=q_config,
#                                  device_map='auto',
#                                  trust_remote_code=True,
#                                  revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_mib = memory_footprint_bytes / (1024 ** 2)  # 转换为 MiB

print(f"{memory_footprint_mib:.2f}MiB")

3739.69MiB


In [19]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training

kbit_model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [20]:
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [21]:
target_modules

['query_key_value']

In [22]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [23]:
qlora_model = get_peft_model(kbit_model, lora_config)

In [24]:
qlora_model.print_trainable_parameters()


trainable params: 974,848 || all params: 6,244,558,848 || trainable%: 0.01561115883009451


In [31]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=f"models/{model_name_or_path}",          # 输出目录
    per_device_train_batch_size=16,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=4,                     # 梯度累积步数
   # per_device_eval_batch_size=8,                      # 每个设备的评估批量大小
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=1,                                # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=100,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=200,                                    # 模型保存步数
    #evaluation_strategy="steps",                       # 评估策略
    #eval_steps=500,                                    # 评估步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)

In [32]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [33]:
trainer.train()

Step,Training Loss
100,3.108200
200,3.106200
300,3.095400
400,3.098900
500,3.082200
600,3.119400
700,3.117300
800,3.095600
900,3.084500
1000,3.057500


Checkpoint destination directory models/ChatGLM3-6b/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory models/ChatGLM3-6b/checkpoint-400 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed expl

TrainOutput(global_step=1790, training_loss=3.0677200573116705, metrics={'train_runtime': 24979.2149, 'train_samples_per_second': 4.588, 'train_steps_per_second': 0.072, 'total_flos': 6.918916981222932e+17, 'train_loss': 3.0677200573116705, 'epoch': 1.0})

In [34]:
trainer.model.save_pretrained(f"models/demo/{model_name_or_path}")

## 模型推理 - 使用 QLoRA 微调后的 ChatGLM3-6B

In [35]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = 'ChatGLM3-6b'  # 模型ID或本地路径
peft_model_path = f"models/demo/{model_name_or_path}"


In [36]:
config = PeftConfig.from_pretrained(peft_model_path)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')
base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [37]:
input_text = '类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领'
print(f'输入：\n{input_text}')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

输入：
类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领


In [38]:
response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调前：\n{response}')

/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


ChatGLM3-6B 微调前：
你好，根据你提供的关键词，我为你找到了以下内容：

1. 显瘦裙：这款裙子采用了版型设计，能有效修饰身材，让身材更加显瘦，适合各种场合穿着。

2. 文艺风裙：这款裙子设计简约，但具有浓郁的文艺风格，穿上它，仿佛能置身于优美的诗句和画意之间。

3. 简约印花裙：这款裙子简约大方，印花图案使裙子更加有活力，穿上它，让人眼前一亮。

4. 撞色裙：这款裙子颜色搭配鲜艳，给人强烈的视觉冲击，让人过目难忘。

5. 压褶连衣裙：这款裙子采用了压褶设计，裙子下摆流畅，穿上它，显得更加优雅。

6. 圆领连衣裙：这款裙子采用了圆领设计，显得更加高贵典雅。

希望这些内容对你有所帮助。如果你还有其他问题，欢迎继续提问。


In [39]:
model = PeftModel.from_pretrained(base_model, peft_model_path)
response, history = model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调后: \n{response}')

ChatGLM3-6B 微调后: 
简约而优雅的连衣裙，采用经典的圆领设计，修饰出纤细修长的脖颈曲线。结合腰间压褶的装饰，勾勒出纤细的腰肢曲线，展现优美的身材曲线。而别致的撞色印花，尽显文艺气质，彰显出别致的设计感。修身的版型，上身更显优雅高挑。
